Obtenemos información sobre nuestra GPU

In [1]:
!nvidia-smi

Sat Nov 18 17:14:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   38C    P8    16W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Instalamos los requerimientos necesarios.

In [2]:
!pip install -r ./requerimientos.txt -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 35.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 2.3 MB/s eta 0:00:00:00:01m0:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 8.4 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu116
    Uninstalling torch-1.12.0+cu116:
      Successfully uninstalled torch-1.12.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu116
    Uninstalling torchvision-0.13.0+cu116:
      Successfully uninstalled torchvision-0.13.0+cu116
  Attempting uninstall: torchaudio
    Found 

Vemos las funciones disponibles con datset_tool.py

In [13]:
!python dataset_tool.py --help

Usage: dataset_tool.py [OPTIONS]

  Convert an image dataset into a dataset archive usable with StyleGAN2 ADA
  PyTorch.

  The input dataset format is guessed from the --source argument:

  --source *_lmdb/                    Load LSUN dataset
  --source cifar-10-python.tar.gz     Load CIFAR-10 dataset
  --source train-images-idx3-ubyte.gz Load MNIST dataset
  --source path/                      Recursively load all images from path/
  --source dataset.zip                Recursively load all images from dataset.zip

  Specifying the output format and path:

  --dest /path/to/dir                 Save output files under /path/to/dir
  --dest /path/to/dataset.zip         Save output files into /path/to/dataset.zip

  The output dataset format can be either an image folder or an uncompressed
  zip archive. Zip archives makes it easier to move datasets around file
  servers and clusters, and may offer better training performance on network
  file systems.

  Images within the dataset archi

Hacemos una prueba para ver que todo vaya bien

In [3]:
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297 --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl


Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl"...
Generating image for seed 85 (0/3) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 265 (1/3) ...
Generating image for seed 297 (2/3) ...


Ahora preparamos las imagenes para ser entrenados con dataset_tool.py

In [4]:
!python dataset_tool.py --source /notebooks/Conjunto_Entrenamiento/ --dest /notebooks/conjunto_entrenamiento/Conjunto_Entrenamiento.zip




100%|█████████████████████████████████████| 13446/13446 [02:15<00:00, 99.51it/s]


funciones disponibles con train.py

In [29]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in the paper "Training Generative
  Adversarial Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU count. Good starting point for new datasets.
    stylegan2  Rep

Es necesario instalar estas dependencias para que funcione

In [3]:
!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jax 0.3.14
Uninstalling jax-0.3.14:
  Successfully uninstalled jax-0.3.14
Found existing installation: jaxlib 0.3.8+cuda11.cudnn82
Uninstalling jaxlib-0.3.8+cuda11.cudnn82:
  Successfully uninstalled jaxlib-0.3.8+cuda11.cudnn82
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 65.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 MB 11.0 MB/s eta 0:00:0000:0100:01
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088051 sha256=d73fa069187bb6c626057d013c97dae892fbc524cbb0af9fdf00b799aa5b5eb9
  Stored in directory: /root/.cache/pip/wheels/14/4a/ff/e9ddfa09012c67d22f926a7873c546c04e722969e8d86f84ec
Successfully built jax


Comenzamos el entrenamiento. El valor de snap es el momento en que hacemos un guardado del entrenamiento y evaluamos las metricas. con augpipe podemos elegir que tipos de transformaciones de aumentos queremos. Con metrics elegimos las metricas que nos interesa evaluar, gamma es el valor de regularizacion, mirror es True si queremos que las imagenes tenga la orientación inversa (como enfrente de un espejo).

In [ ]:
!python train.py --outdir resultados_nuevos --data /notebooks/conjunto_entrenamiento/Conjunto_Entrenamiento.zip --gpus=1 --snap=100 --augpipe=bg --metrics=fid50k_full,kid50k_full --gamma=25 --mirror=True --resume=ffhq256



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 100,
  "network_snapshot_ticks": 100,
  "metrics": [
    "fid50k_full",
    "kid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/notebooks/conjunto_entrenamiento/Conjunto_Entrenamiento.zip",
    "use_labels": false,
    "max_size": 13446,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 

In [1]:
!python generate.py --help

Usage: generate.py [OPTIONS]

  Generate images using pretrained network pickle.

  Examples:

  # Generate curated MetFaces images without truncation (Fig.10 left)
  python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

  # Generate uncurated MetFaces images with truncation (Fig.12 upper left)
  python generate.py --outdir=out --trunc=0.7 --seeds=600-605 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

  # Generate class conditional CIFAR-10 images (Fig.17 left, Car)
  python generate.py --outdir=out --seeds=0-35 --class=1 \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/cifar10.pkl

  # Render an image from projected W
  python generate.py --outdir=out --projected_w=projected_w.npz \
      --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

Options:
  --net

Ahora vamos a generar las imagenes falsas

In [4]:
!python generate.py --outdir=imagenes_generadas_trunc1 --trunc=0.5 --seeds=0-500 --network=resultados_nuevos/00001-Conjunto_Entrenamiento-mirror-auto1-gamma25-bg-resumeffhq256/network-snapshot-000800.pkl

Loading networks from "resultados_nuevos/00001-Conjunto_Entrenamiento-mirror-auto1-gamma25-bg-resumeffhq256/network-snapshot-001200.pkl"...
Generating image for seed 0 (0/16001) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/16001) ...
Generating image for seed 2 (2/16001) ...
Generating image for seed 3 (3/16001) ...
Generating image for seed 4 (4/16001) ...
Generating image for seed 5 (5/16001) ...
Generating image for seed 6 (6/16001) ...
Generating image for seed 7 (7/16001) ...
Generating image for seed 8 (8/16001) ...
Generating image for seed 9 (9/16001) ...
Generating image for seed 10 (10/16001) ...
Generating image for seed 11 (11/16001) ...
Generating image for seed 12 (12/16001) ...
Generating image for seed 13 (13/16001) ...
Generating image for seed 14 (14/16001) ...
Generating image for seed 15 (15/16001) ...
Generating image for seed 16 (16/16001) ...
Generating image for seed

In [4]:
import os

trunc_values = [0.5, 0.7, 0.8, 1.0, 1.3, 1.5]

for trunc in trunc_values:
    command = f"python generate.py --outdir=imagenes_generadas_trunc{trunc} --trunc={trunc} --seeds=0-500 --network=resultados_nuevos/00001-Conjunto_Entrenamiento-mirror-auto1-gamma25-bg-resumeffhq256/network-snapshot-000800.pkl"
    os.system(command)


Loading networks from "resultados_nuevos/00001-Conjunto_Entrenamiento-mirror-auto1-gamma25-bg-resumeffhq256/network-snapshot-000800.pkl"...
Generating image for seed 0 (0/501) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/501) ...
Generating image for seed 2 (2/501) ...
Generating image for seed 3 (3/501) ...
Generating image for seed 4 (4/501) ...
Generating image for seed 5 (5/501) ...
Generating image for seed 6 (6/501) ...
Generating image for seed 7 (7/501) ...
Generating image for seed 8 (8/501) ...
Generating image for seed 9 (9/501) ...
Generating image for seed 10 (10/501) ...
Generating image for seed 11 (11/501) ...
Generating image for seed 12 (12/501) ...
Generating image for seed 13 (13/501) ...
Generating image for seed 14 (14/501) ...
Generating image for seed 15 (15/501) ...
Generating image for seed 16 (16/501) ...
Generating image for seed 17 (17/501) ...
Generating image 

In [5]:
pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [6]:
from PIL import Image
import os

def combine_images_from_folder(folder, output_filename, images_per_row=10):
    images = [Image.open(os.path.join(folder, file)) for file in os.listdir(folder) if file.endswith('.png')]
    if not images:
        print(f"No images found in folder {folder}")
        return

    # Asumiendo que todas las imágenes tienen el mismo tamaño
    image_width, image_height = images[0].size
    combined_image_width = image_width * images_per_row
    combined_image_height = image_height * (len(images) // images_per_row + 1)
    
    combined_image = Image.new('RGB', (combined_image_width, combined_image_height))

    for i, image in enumerate(images):
        x_offset = (i % images_per_row) * image_width
        y_offset = (i // images_per_row) * image_height
        combined_image.paste(image, (x_offset, y_offset))

    combined_image.save(output_filename)

trunc_values = [0.5, 0.7, 0.8, 1.0, 1.3, 1.5]
for trunc in trunc_values:
    folder = f'imagenes_generadas_trunc{trunc}'
    output_filename = f'combined_trunc{trunc}.png'
    combine_images_from_folder(folder, output_filename)
